In [1]:
import pandas as pd
import camelot
import numpy as np

In [2]:
pfad = "Lieferung_SEB-Schulergebnisse_2/Schulberichte/AlleSchulen_2009.pdf"
umfrage = camelot.read_pdf(pfad, flavor='stream', pages='all')

Ergebnisse der Exploration:
- 24 Seiten, 24 Tabellen, diesmal passts
- Die Zahlen sind erneut als Strings erfasst
- es hat wieder Tabellen, die verschoben sind: dort fehlen 2 Zeilen (wenn bei loc 1,13 nicht "Beurteilung" steht) 
    - Ausnahmen:
        ZO, dort steht der Wert bei 1,12
        Tabelle 9, Katholisches Gymnasium: Dort hats eine Zeile zu viel; Beurteilung ist auf 2,13
- die gesuchten Daten sind an folgenden Orten:
    "name": 2,0, 
    "zufr": 6,4, 
    "vorb_f": 7,4
    "vorb_p":9,4
    "prob": 12,4 
    "urt": 13,4 
    "zus": 16,4 
    "konf": 17,4


In [8]:
umfrage[0].df.loc[range(10,20)]

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
10,Studium (5),,,,,,,,,,...,,,,,,,,,,
11,,insgesamt,,58,4.3,4.3,0.1,,1,2,...,18,22,6,2,3,16,31,38,10,79.3
12,,selbständig Probleme zu lösen.,,64,4.7,4.4,0.3,,1,2,...,14,31,12,2,3,6,22,48,19,89.1
13,,eigenständig zu urteilen.,,64,4.8,4.6,0.2,,1,3,...,9,33,15,2,5,5,14,52,23,89.1
14,,zielorientiert zu handeln.,,64,4.6,4.5,0.2,,2,3,...,11,30,13,3,5,8,17,47,20,84.4
15,Persönliche \nEntwicklung (1)\ngelernt . . .,neues Wissen zu erwerben.,,64,5.5,5.3,0.2,,0,0,...,4,24,35,0,0,2,6,38,55,98.4
16,,in Gruppen zu arbeiten.,,64,4.3,4.3,0.0,,0,5,...,14,32,4,0,8,14,22,50,6,78.1
17,,mit Konflikten umzugehen.,,63,4.2,3.9,0.4,*,0,6,...,22,22,6,0,10,11,35,35,10,79.4
18,,mich in der heutigen Welt zurechtzufinden.,,62,4.0,3.8,0.2,,4,4,...,16,23,3,6,6,19,26,37,5,67.7
19,,soziale Verantwortung zu übernehmen.,,63,3.8,4.0,-0.2,,5,2,...,17,14,5,8,3,32,27,22,8,57.1


Daraus ergibt sich folgende Anpassung des if-Argumentes:

In [9]:
for table in range(0, len(umfrage)):
    df_schule = umfrage[table].df
    
    if df_schule.loc[1,13] != "Beurteilung" and df_schule.loc[1,12] != "Beurteilung":   #Wir checken hier anhand eines eindeutigen Merkmals, ob eine Zeile fehlt
        df_newrow = df_schule.head(2)
        df_schule = pd.concat([df_newrow, df_schule], ignore_index = True)   # Ignore Index ist entscheiden!              
    schulname = df_schule.loc[2,0]
    print(schulname)

Büelrain
Enge
Freies Gymnasium Zürich
Freudenberg
Glattal
Gymnasium Unterstrass
Hohe Promenade
Hottingen
Im Lee
Katholisches
KME
Küsnacht
Liceo Artistico
Limmattal
Oerlikon
Rämibühl LG
Rämibühl MNG
Rämibühl RG
Rychenberg
Stadelhofen
Wiedikon
Zürcher Oberland
Zürcher Unterland
Zürich Birch


Das hier müssen wir bei der Ausnahme 9 machen: eine Reihe droppen und den Index resetten (Drop = True heisst: ohne den alten als Spalte zu behalten)
Das bauen wir dann in ein eigenes if-Argument ganz am Anfang ein.

In [10]:
umfrage[9].df.drop(0).reset_index(drop=True).head(10)

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,,,,,,Mittelwert,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,Beurteilung,,,,,,,Anteil
2,Gymnasium Zürich,,,,,der Beurteilung,,,,,...,,,,,,,,,,
3,,,,Antworten,diese,,Abweichung,,,negativ positiv,...,,,,,negativ positiv,,,,,der positiven
4,,,Anzahl,,Gruppe,alle,zu allen,,,,...,,,,,,Prozent,,,,Bewertungen
5,Frage (Nr.),,,,2009,2009,anderen,,1,2,...,4,5,6,1,2,3,4,5,6,(%)
6,(6),Zufriedenheit mit der Mittelschule,,11,4.6,4.9,-0.2,,0,1,...,1,6,2,0,9,9,9,55,18,81.8
7,,in fachlicher Hinsicht,,7,4.3,4.2,0.1,,0,0,...,5,2,0,0,0,0,71,29,0,100.0
8,Vorbereitung,,,,,,,,,,...,,,,,,,,,,
9,,in persönlichkeitsbildender Hinsicht,,8,3.9,4.3,-0.4,,2,0,...,3,1,2,25,0,0,38,13,25,75.0


In [12]:
jahr = 2009
dic_list = []


for table in range(0, len(umfrage)):
    df_schule = umfrage[table].df
    
    if df_schule.loc[1,13] != "Beurteilung" and df_schule.loc[1,12] != "Beurteilung":   #Wir checken hier anhand eines eindeutigen Merkmals, ob eine Zeile fehlt
        df_newrow = df_schule.head(2)
        df_schule = pd.concat([df_newrow, df_schule], ignore_index = True)   # Ignore Index ist entscheiden!              
        
    name = df_schule.loc[2,0]
    
    df_kurz = df_schule.drop([0,1,2,3,4,5])
    print (name + str(df_kurz.loc[6,4]))

Büelrain4.9
Enge4.8
Freies Gymnasium Zürich4.6
Freudenberg4.6
Glattal4.7
Gymnasium Unterstrass5.0
Hohe Promenade4.8
Hottingen5.0
Im Lee4.7
Katholischesdiese
KME4.9
Küsnacht5.3
Liceo Artistico4.8
Limmattal4.9
Oerlikon4.8
Rämibühl LG4.7
Rämibühl MNG4.8
Rämibühl RG4.9
Rychenberg4.9
Stadelhofen5.0
Wiedikon4.5
Zürcher Oberland4.9
Zürcher Unterland5.0
Zürich Birch4.7


In [13]:
jahr = 2009
dic_list = []


for table in range(0, len(umfrage)):
    df_schule = umfrage[table].df
    
    if df_schule.loc[2,13] == "Beurteilung":
        df_schule = df_schule.drop(0).reset_index(drop=True)  
    
    if df_schule.loc[1,13] != "Beurteilung" and df_schule.loc[1,12] != "Beurteilung":   #Wir checken hier anhand eines eindeutigen Merkmals, ob eine Zeile fehlt
        df_newrow = df_schule.head(2)
        df_schule = pd.concat([df_newrow, df_schule], ignore_index = True)   # Ignore Index ist entscheiden!              
        
    name = df_schule.loc[2,0]
    
    df_kurz = df_schule.drop([0,1,2,3,4,5])
    df_kurz = df_kurz.replace("", np.nan)
    df_kurz[4] = df_kurz[4].astype(float)
        
    zufr = df_kurz.loc[6,4]
    vorb_f = df_kurz.loc[7,4]
    vorb_p = df_kurz.loc[9,4]
    prob = df_kurz.loc[12,4]
    urt = df_kurz.loc[13,4]
    zus = df_kurz.loc[16,4]
    konf = df_kurz.loc[17,4]

    mini_dict = {"jahr": jahr, 
                 "name": name, 
                 "zufr": zufr, 
                 "vorb_f": vorb_f, 
                 "vorb_p":vorb_p, 
                 "prob": prob, 
                 "urt": urt, 
                 "zus": zus, 
                 "konf": konf}
    dic_list.append(mini_dict)

In [14]:
Resultat = pd.DataFrame(dic_list)
Resultat

,jahr,name,zufr,vorb_f,vorb_p,prob,urt,zus,konf
0,2009,Büelrain,4.9,4.4,4.4,4.7,4.8,4.3,4.2
1,2009,Enge,4.8,4.2,4.1,4.5,4.7,4.1,3.7
2,2009,Freies Gymnasium Zürich,4.6,3.5,3.9,4.0,4.6,3.9,3.8
3,2009,Freudenberg,4.6,4.4,4.1,4.5,4.5,4.4,3.5
4,2009,Glattal,4.7,4.2,4.2,4.4,4.7,4.2,3.9
5,2009,Gymnasium Unterstrass,5.0,3.6,4.8,4.5,4.7,4.6,4.2
6,2009,Hohe Promenade,4.8,4.5,4.5,4.2,4.2,4.2,4.0
7,2009,Hottingen,5.0,4.0,4.2,4.3,4.6,4.5,4.1
8,2009,Im Lee,4.7,4.4,4.3,4.5,4.5,4.1,3.8
9,2009,Gymnasium Zürich,4.6,4.3,3.9,4.8,5.2,4.9,3.9
